In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.7/182.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/

In [ ]:
!pip install --no-deps xformers  trl peft accelerate bitsandbytes WandB

In [ ]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sachinmosambe7814 (sachinmosambe7814-iit-kharagpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
# Set dtype based on bfloat16 support
dtype = torch.float16 if not torch.cuda.is_bf16_supported() else torch.bfloat16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3",
    max_seq_length = max_seq_length,
    dtype = dtype, # Use determined dtype
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.7: Fast Mistral patching. Transformers: 4.55.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = False, # Disabled gradient checkpointing
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from datasets import load_dataset, DatasetDict

# Define your prompt formatting function (example)
alpaca_prompt = """Below is an instruction describing your role, paired with an input that provides detailed context about the task. Write a response that appropriately completes the request.

### Instructions:
{}

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}



# Load full dataset
full_dataset = load_dataset("json", data_files="/content/cleaned_combined_dataset.jsonl")

# Split 80/20 train/validation
splits = full_dataset["train"].train_test_split(test_size=0.2, seed=3407)
dataset = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"]
})

# Format prompts on both splits
dataset = dataset.map(formatting_prompts_func, batched=True)

# Save to disk for later use
dataset["train"].to_json("train_dataset.jsonl")
dataset["validation"].to_json("validation_dataset.jsonl")


Creating json from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

15055630

In [ ]:
from datasets import load_dataset, DatasetDict

# Load full dataset from JSONL file
full_dataset = load_dataset("json", data_files="/content/cleaned_combined_dataset.jsonl")["train"]

# Step 1: Split 80% train, 20% temp (validation + test)
train_test_split = full_dataset.train_test_split(test_size=0.2, seed=3407)

# Step 2: Split the 20% temp into 50% validation, 50% test → effectively 10% val, 10% test
val_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=3407)

# Combine splits in a DatasetDict
dataset = DatasetDict({
    "train": train_test_split["train"],
    "validation": val_test_split["train"],
    "test": val_test_split["test"],
})

print(dataset)


In [ ]:
# Assuming formatting_prompts_func is defined and ready

dataset = dataset.map(formatting_prompts_func, batched=True)

dataset["train"].to_json("train_dataset.jsonl")
dataset["validation"].to_json("validation_dataset.jsonl")
dataset["test"].to_json("test_dataset.jsonl")


In [ ]:
train_dataset = load_dataset("json", data_files="train_dataset.jsonl", split="train")
validation_dataset = load_dataset("json", data_files="validation_dataset.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        max_steps=60,
        eval_steps=10,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="wandb",
    )
)

trainer.train()

trainer.save_model("outputs/final_model")
tokenizer.save_pretrained("outputs/final_model")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from evaluate import load
import torch
from tqdm import tqdm



# Load your fine-tuned model and tokenizer
model_path = "outputs/merged_full_model"  # Update this to your saved model path
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Define your alpaca prompt template and EOS token
alpaca_prompt = """### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Load your test dataset
test_dataset = load_dataset("json", data_files="test_dataset.jsonl", split="train")

# Build prompt for evaluation (without the output)
def build_eval_prompt(instruction, input_text):
    return f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"

# Generate predictions
predictions = []
references = []

for sample in tqdm(test_dataset):
    prompt = build_eval_prompt(sample["instruction"], sample["input"])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        num_beams=5,
        early_stopping=True,
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract text after "Response:"
    generated_answer = generated_text.split("Response:")[-1].strip()

    predictions.append(generated_answer)
    references.append(sample["output"])

# Load evaluation metrics
metric_em = load("exact_match")
metric_f1 = load("f1")

# Compute metrics
em_score = metric_em.compute(predictions=predictions, references=references)
f1_score = metric_f1.compute(predictions=predictions, references=references)

print(f"Exact Match: {em_score['exact_match']:.2f}")
print(f"F1 Score: {f1_score['f1']:.2f}")


In [ ]:
merged_model = PeftModel.merge_and_unload(model)

# Save merged model
merged_model.save_pretrained("outputs/merged_full_model")
tokenizer.save_pretrained("outputs/merged_full_model")


In [ ]:
from google.colab import files
import shutil

# Zip the entire model folder
shutil.make_archive("merged_full_model", 'zip', "outputs/merged_full_model")

# Download the zipped file
files.download("merged_full_model.zip")
